# SciTeX Scholar Tutorial

This notebook demonstrates how to use the `scitex.scholar` module for scientific literature management.

## Features Covered
- Literature search from multiple sources
- Paper metadata enrichment with impact factors
- BibTeX generation
- PDF downloads (when available)
- Building a local search index

## 1. Basic Literature Search

Let's start with a simple search for papers:

In [ ]:
import asyncio
from scitex.scholar import search_papers, Paper

# Search for papers
papers = await search_papers(
    query="deep learning neuroscience",
    limit=5
)

# Display results
for i, paper in enumerate(papers, 1):
    print(f"{i}. {paper.title}")
    print(f"   Authors: {', '.join(paper.authors[:2])}...")
    print(f"   Year: {paper.year}, Citations: {paper.citation_count}")
    print()

## 2. Creating an Enriched Bibliography

Add journal impact factors and other metrics to your bibliography:

In [ ]:
from scitex.scholar import PaperEnrichmentService

# Enrich papers with journal metrics
enricher = PaperEnrichmentService()
enriched_papers = enricher.enrich_papers(papers)

# Show enriched data
for paper in enriched_papers:
    if paper.impact_factor:
        print(f"📊 {paper.journal}: IF={paper.impact_factor}")

## 3. Generate BibTeX Entries

Create properly formatted BibTeX entries with enriched metadata:

In [ ]:
# Generate BibTeX for the first paper
if papers:
    bibtex = papers[0].to_bibtex(include_enriched=True)
    print(bibtex)

## 4. Save Complete Bibliography

In [ ]:
from pathlib import Path
from scitex.scholar import generate_enriched_bibliography

# Save all papers to a BibTeX file
output_path = Path("my_bibliography.bib")
generate_enriched_bibliography(
    enriched_papers,
    output_path,
    enrich=False  # Already enriched
)

print(f"Bibliography saved to: {output_path}")

## 5. Advanced Search with AI Enhancement

Use AI to improve search results (requires API keys):

In [ ]:
from scitex.scholar import PaperAcquisition

# Initialize with AI enhancement
acquisition = PaperAcquisition(
    use_ai=True,  # Enable AI features
    email="your.email@example.com"  # Required for some APIs
)

# Search with specific criteria
results = await acquisition.search(
    query="phase amplitude coupling cognitive control",
    sources=['semantic_scholar', 'pubmed'],
    max_results=10,
    start_year=2020
)

print(f"Found {len(results)} recent papers on the topic")

## 6. Building a Local Search Index

Create a searchable index of your PDF collection:

In [ ]:
from scitex.scholar import build_index, search_sync

# Build index from PDFs in a directory
pdf_directory = Path("~/Documents/Papers").expanduser()
if pdf_directory.exists():
    build_index(str(pdf_directory))
    
    # Search your local collection
    local_results = search_sync(
        "neural oscillations",
        local_only=True
    )
    
    print(f"Found {len(local_results)} papers in local collection")

## Tips and Best Practices

1. **Rate Limiting**: The module automatically handles rate limiting for APIs
2. **Caching**: Search results are cached to avoid redundant API calls
3. **Error Handling**: Network errors are handled gracefully
4. **PDF Access**: Only open-access PDFs can be downloaded automatically

## Environment Variables

Set these for enhanced functionality:
- `OPENAI_API_KEY`: For AI-enhanced search
- `SEMANTIC_SCHOLAR_API_KEY`: For higher rate limits
- `ENTREZ_EMAIL`: Your email for PubMed API